In [1]:
#Refrence material 

https://www.coursera.org/learn/unsupervised-learning-recommenders-reinforcement-learning/programming/Rqh9M/anomaly-detection/lab?path=%2Fnotebooks%2FC3_W1_Anomaly_Detection.ipynb

In [1]:
import pandas as pd
import numpy as np

In [8]:
df=pd.read_csv('final_train_data.csv')

In [9]:
df.head(5)

,id,Product ID,Type,Air temperature [K],Process temperature [K],Rotational speed [rpm],Torque [Nm],Tool wear [min],Machine failure,TWF,HDF,PWF,OSF,RNF
0,0,L50096,L,300.6,309.6,1596,36.1,140,0,0,0,0,0,0
1,1,M20343,M,302.6,312.1,1759,29.1,200,0,0,0,0,0,0
2,2,L49454,L,299.3,308.5,1805,26.5,25,0,0,0,0,0,0
3,3,L53355,L,301.0,310.9,1524,44.3,197,0,0,0,0,0,0
4,4,M24050,M,298.0,309.0,1641,35.4,34,0,0,0,0,0,0


In [10]:
df=df.drop(['id','Product ID','Machine failure','TWF','HDF','PWF','OSF','RNF','Type'], axis=1)

In [11]:
df.head(5)

,Air temperature [K],Process temperature [K],Rotational speed [rpm],Torque [Nm],Tool wear [min]
0,300.6,309.6,1596,36.1,140
1,302.6,312.1,1759,29.1,200
2,299.3,308.5,1805,26.5,25
3,301.0,310.9,1524,44.3,197
4,298.0,309.0,1641,35.4,34


In [6]:
# df['Type'] = df['Type'].replace({'H':0,'M':1,'L':2})

In [7]:
# df.head(5)

,Type,Air temperature [K],Process temperature [K],Rotational speed [rpm],Torque [Nm],Tool wear [min],TWF,HDF,PWF,OSF,RNF
0,2,300.6,309.6,1596,36.1,140,0,0,0,0,0
1,1,302.6,312.1,1759,29.1,200,0,0,0,0,0
2,2,299.3,308.5,1805,26.5,25,0,0,0,0,0
3,2,301.0,310.9,1524,44.3,197,0,0,0,0,0
4,1,298.0,309.0,1641,35.4,34,0,0,0,0,0


In [12]:
df.shape  #We have 11 features

(132133, 5)

In [9]:
# df=df.drop(['Type'], axis=1)

In [10]:
# df.head(5)

,Air temperature [K],Process temperature [K],Rotational speed [rpm],Torque [Nm],Tool wear [min],TWF,HDF,PWF,OSF,RNF
0,300.6,309.6,1596,36.1,140,0,0,0,0,0
1,302.6,312.1,1759,29.1,200,0,0,0,0,0
2,299.3,308.5,1805,26.5,25,0,0,0,0,0
3,301.0,310.9,1524,44.3,197,0,0,0,0,0
4,298.0,309.0,1641,35.4,34,0,0,0,0,0


In [83]:
# df['TWF'].value_counts()

0    132133
Name: TWF, dtype: int64

In [11]:
# df.sample(30000,random_state=1)  #Taking random 30k values

In [12]:
#converting dataframe to array

In [13]:
x = df.to_numpy()

In [14]:
x.shape

(132133, 5)

In [15]:
def estimate_gaussian(X):
#     X -> (m,n) array -> m =examples and n=attributes
#     returns : mu -> mean and var -> variance ; (n dimensional arrays) -> (n,)
    #code:
    
    m,n=X.shape
    mu_=(1/m)*np.sum(X,axis=0)
    mu=mu_.T
    var_=np.sum(((X-mu)**2),axis=0)/m
    var=var_.T
    
    return mu,var
        
        


In [16]:
mu,var=estimate_gaussian(x)

In [17]:
mu.shape

(5,)

In [18]:
mu

array([ 299.8474405 ,  309.93606442, 1521.46349511,   40.18705494,
        103.93653364])

In [19]:
var.shape

(5,)

In [20]:
var

array([3.44398299e+00, 1.92335262e+00, 1.82080950e+04, 6.90629176e+01,
       4.05841599e+03])

In [21]:
#Multivariance gussian
def multivariance_gussian(X,mu,var):
#     X -> (m,n) array -> m =examples and n=attributes
#     mu -> (ndarary,)
#     var -> (ndarray,)
    
    #code:
    n_features=X.shape[1]
    #constant = 1 / ((2 * np.pi) ** (n_features / 2) * np.sqrt(np.prod(var+(1e-10))))
    constant=1
    # Compute the exponent term of the Gaussian function
    exponent = -np.sum(((X - mu) ** 2) / (var+(1e-10)), axis=1)
    
    # Compute the probabilities for each sample
    probabilities = constant * np.exp(exponent)
    
    return probabilities
    

In [22]:
#Getting mean and variance of the dataset
mu,var=estimate_gaussian(x)

In [23]:
p=multivariance_gussian(x,mu,var)

In [24]:
p.shape

(132133,)

In [25]:
#Selecting an epsilon for detecting threshold
def select_threshold(y_val, p_val): 
    """
    Finds the best threshold to use for selecting outliers 
    based on the results from a validation set (p_val) 
    and the ground truth (y_val)
    
    Args:
        y_val (ndarray): Ground truth on validation set
        p_val (ndarray): Results on validation set
        
    Returns:
        epsilon (float): Threshold chosen 
        F1 (float):      F1 score by choosing epsilon as threshold
    """ 

    best_epsilon = 0
    best_F1 = 0
    F1 = 0
    
    step_size = (max(p_val) - min(p_val)) / 1000
    
    for epsilon in np.arange(min(p_val), max(p_val), step_size):
    
        ### START CODE HERE ### 
        predictions = (p_val < epsilon)
        tp = np.sum((predictions == 1) & (y_val == 1))
        fn = np.sum((predictions == 0) & (y_val == 1))
        fp = sum((predictions == 1) & (y_val == 0))
        prec = tp / (tp + fp)
        rec = tp / (tp + fn)
        F1 = 2 * prec * rec / (prec + rec)
        
        
        
        
        
        
        ### END CODE HERE ### 
        
        if F1 > best_F1:
            best_F1 = F1
            best_epsilon = epsilon
        
    return best_epsilon, best_F1

In [26]:
df_cv=pd.read_csv('final_cv_data_.csv')

In [27]:
df_cv.head(5)

,id,Product ID,Type,Air temperature [K],Process temperature [K],Rotational speed [rpm],Torque [Nm],Tool wear [min],Machine failure,TWF,HDF,PWF,OSF,RNF
0,90418,L49271,L,299.5,309.2,1399,47.4,20,0,0,0,0,0,0
1,103002,L53040,L,301.3,311.0,1497,45.5,84,0,0,0,0,0,0
2,9653,L54424,L,299.8,309.7,1472,42.3,193,0,0,0,0,0,0
3,106459,M16074,M,296.8,307.8,1419,47.3,0,0,0,0,0,0,0
4,133925,L50355,L,300.3,309.6,1488,39.7,82,0,0,0,0,0,0


In [28]:
df_cv['Machine failure'].value_counts()

0    1157
1    1139
Name: Machine failure, dtype: int64

In [29]:
y = df_cv['Machine failure']

In [30]:
y = y.to_numpy()

In [31]:
y.shape

(2296,)

In [33]:
df_cv=df_cv.drop(['id','Product ID','Machine failure','TWF','HDF','PWF','OSF','RNF','Type'], axis=1)

In [34]:
df_cv.head(5)

,Air temperature [K],Process temperature [K],Rotational speed [rpm],Torque [Nm],Tool wear [min]
0,299.5,309.2,1399,47.4,20
1,301.3,311.0,1497,45.5,84
2,299.8,309.7,1472,42.3,193
3,296.8,307.8,1419,47.3,0
4,300.3,309.6,1488,39.7,82


In [35]:
x_cv = df_cv.to_numpy()

In [36]:
x_cv.shape

(2296, 5)

In [37]:
# y->   ground truth labels array
# x_cv->cross validation dataset array

In [38]:
p_val=multivariance_gussian(x_cv,mu,var)

In [39]:
p_val.shape

(2296,)

In [40]:
p_val

array([2.65239305e-02, 1.75397530e-01, 1.12683339e-01, ...,
       1.49324315e-04, 3.07699738e-02, 1.74237363e-03])

In [41]:
best_epsilon, best_F1=select_threshold(y, p_val)

C:\Users\hrith\AppData\Local\Temp\ipykernel_24900\251301137.py:30: RuntimeWarning: invalid value encountered in long_scalars
  prec = tp / (tp + fp)


In [42]:
best_epsilon

0.013399584845001979

In [43]:
outliers = p_val < best_epsilon

In [44]:
outliers =outliers.astype(int)

In [45]:
outliers.shape

(2296,)

In [46]:
#Counting the number of correct predictions :
count_correct=0
for i in range(len(outliers)):
    if outliers[i]==y[i]:
       count_correct=count_correct+1

In [47]:
print(count_correct)   #Correct Precictions

1699


In [48]:
y.shape    #Total Predictions

(2296,)

In [49]:
# Detected 1669 outoff 2296 cases correctly for cross validation set

In [50]:
c=0
for i in range(len(y)):
    if y[i]==1:
        c=c+1
    

In [51]:
print(c)

1139


In [52]:
# 1139 -> not working

In [53]:
#Counting when machine was faulty and was detected as faulty
count_faulty_correct=0
for i in range(len(outliers)):
    if outliers[i]==1 and y[i]==1:
        count_faulty_correct=count_faulty_correct+1

In [54]:
print(count_faulty_correct)   #Not working engines detected

1040


In [55]:
# Detected 1040 outoff 1139 not working engines correctly for cross validation set

In [57]:
1669/2296   #Droping those columns didnt halp help at all

0.7269163763066202

86.32% accuracy